In [1]:
# -----------------------------
# CNN-1D Regression with TensorFlow/Keras (VS Code Compatible)
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# Load CSV files (Manually provide path)
# -----------------------------
train_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/train_ks.csv"
test_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/test_ks.csv"    

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# -----------------------------
# Features and target
# -----------------------------
target_column = "Yield"
X_train = train_df.drop(columns=[target_column]).values
y_train = train_df[target_column].values
X_test = test_df.drop(columns=[target_column]).values
y_test = test_df[target_column].values

# -----------------------------
# Standardize features
# -----------------------------
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Reshape for 1D-CNN input
X_train_scaled = X_train_scaled[..., np.newaxis]
X_test_scaled = X_test_scaled[..., np.newaxis]

# -----------------------------
# Build 1D-CNN model
# -----------------------------
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1],1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Conv1D(128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

# -----------------------------
# Compile model
# -----------------------------
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# -----------------------------
# EarlyStopping
# -----------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# -----------------------------
# Train model
# -----------------------------
history = model.fit(
    X_train_scaled, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Save model
# -----------------------------
model.save("CNN1D_model.h5")
print("Model saved as CNN1D_model.h5")

# -----------------------------
# Predictions
# -----------------------------
y_pred_train = model.predict(X_train_scaled).flatten()
y_pred_test = model.predict(X_test_scaled).flatten()

# -----------------------------
# Evaluation
# -----------------------------
def evaluate(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    rpd = np.std(y_true, ddof=1) / rmse
    return r2, rmse, rpd

train_r2, train_rmse, train_rpd = evaluate(y_train, y_pred_train)
test_r2, test_rmse, test_rpd = evaluate(y_test, y_pred_test)

print("\n1D-CNN Model Evaluation :")
print(f"Train R²   : {train_r2:.4f}")
print(f"Test R²    : {test_r2:.4f}")
print(f"Train RMSE : {train_rmse:.4f}")
print(f"Test RMSE  : {test_rmse:.4f}")
print(f"Train RPD  : {train_rpd:.4f}")
print(f"Test RPD   : {test_rpd:.4f}")

# -----------------------------
# Scatter Plot: Train vs Test
# -----------------------------
def plot_train_test(y_train, y_pred_train, y_test, y_pred_test):
    fig, axes = plt.subplots(1, 2, figsize=(12,6))

    # Training Plot
    axes[0].scatter(y_train, y_pred_train, color="blue", alpha=0.7, label="Predicted")
    z_train = np.polyfit(y_train, y_pred_train, 1)
    p_train = np.poly1d(z_train)
    axes[0].plot(y_train, p_train(y_train), "r--", label="Best Fit")
    axes[0].plot(y_train, y_train, "g-", label="Ideal Fit")
    axes[0].set_title("Training Set: Predicted vs Actual")
    axes[0].set_xlabel("Actual Values")
    axes[0].set_ylabel("Predicted Values")
    axes[0].legend()
    axes[0].grid(False)

    # Test Plot
    axes[1].scatter(y_test, y_pred_test, color="orange", alpha=0.7, label="Predicted")
    z_test = np.polyfit(y_test, y_pred_test, 1)
    p_test = np.poly1d(z_test)
    axes[1].plot(y_test, p_test(y_test), "r--", label="Best Fit")
    axes[1].plot(y_test, y_test, "g-", label="Ideal Fit")
    axes[1].set_title("Test Set: Predicted vs Actual")
    axes[1].set_xlabel("Actual Values")
    axes[1].set_ylabel("Predicted Values")
    axes[1].legend()
    axes[1].grid(False)

    plt.tight_layout()
    plt.show()

plot_train_test(y_train, y_pred_train, y_test, y_pred_test)

# -----------------------------
# Confusion Matrix (regression → classification)
# -----------------------------
y_test_class = (y_test >= np.median(y_test)).astype(int)
y_pred_test_class = (y_pred_test >= np.median(y_test)).astype(int)

cm = confusion_matrix(y_test_class, y_pred_test_class)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Low Yield", "High Yield"])
plt.figure(figsize=(6,6))
disp.plot(cmap=plt.cm.Blues, values_format="d")
plt.title("Confusion Matrix (Regression → Classification)")
plt.show()


Epoch 1/30
49/49 [==============================] - 5s 52ms/step - loss: 8.1351 - mae: 2.0156 - val_loss: 69.4853 - val_mae: 8.3031
Epoch 2/30
49/49 [==============================] - 2s 46ms/step - loss: 1.7460 - mae: 1.0517 - val_loss: 198.6113 - val_mae: 14.0746
Epoch 3/30
49/49 [==============================] - 2s 47ms/step - loss: 1.3536 - mae: 0.9290 - val_loss: 341.3968 - val_mae: 18.4608
Epoch 4/30
49/49 [==============================] - 2s 47ms/step - loss: 1.0526 - mae: 0.8146 - val_loss: 419.9940 - val_mae: 20.4720
Epoch 5/30
49/49 [==============================] - 2s 47ms/step - loss: 1.0458 - mae: 0.8215 - val_loss: 446.2774 - val_mae: 21.1005
Epoch 6/30
49/49 [==============================] - 2s 46ms/step - loss: 0.9361 - mae: 0.7766 - val_loss: 422.3678 - val_mae: 20.5134
Epoch 7/30
49/49 [==============================] - 2s 46ms/step - loss: 0.9487 - mae: 0.7725 - val_loss: 359.3546 - val_mae: 18.9062
Epoch 8/30
49/49 [==============================] - 2s 47ms/step

KeyboardInterrupt: 

In [3]:
# -----------------------------
# 1D-CNN Regression (GPU-ready)
# -----------------------------
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, ConfusionMatrixDisplay

# -----------------------------
# GPU Setup (4GB safe)
# -----------------------------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected. Using CPU.")

# -----------------------------
# Load dataset
# -----------------------------
train_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/train_ks.csv"
test_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/test_ks.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

target_column = "Yield"
X_train = train_df.drop(columns=[target_column]).values
y_train = train_df[target_column].values
X_test = test_df.drop(columns=[target_column]).values
y_test = test_df[target_column].values

# -----------------------------
# Standardize features
# -----------------------------
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

X_train_scaled = X_train_scaled[..., np.newaxis]
X_test_scaled = X_test_scaled[..., np.newaxis]

# -----------------------------
# Build 1D-CNN Model
# -----------------------------
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.2),

    Conv1D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train_scaled, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Save model
# -----------------------------
model.save("CNN1D_final_model.keras")
print("Model saved as CNN1D_final_model.keras")

# -----------------------------
# Reuse saved model
# -----------------------------
model_loaded = load_model("CNN1D_final_model.keras")
scaler_loaded = pickle.load(open("scaler_X.pkl", "rb"))

X_train_scaled_loaded = scaler_loaded.transform(X_train)[..., np.newaxis]
X_test_scaled_loaded = scaler_loaded.transform(X_test)[..., np.newaxis]

y_pred_train = model_loaded.predict(X_train_scaled_loaded).flatten()
y_pred_test = model_loaded.predict(X_test_scaled_loaded).flatten()

# -----------------------------
# Evaluation function
# -----------------------------
def evaluate(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    rpd = np.std(y_true, ddof=1) / rmse
    return r2, rmse, rpd

train_r2, train_rmse, train_rpd = evaluate(y_train, y_pred_train)
test_r2, test_rmse, test_rpd = evaluate(y_test, y_pred_test)

print("\nLoaded 1D-CNN Model Evaluation:")
print(f"Train R²   : {train_r2:.4f}")
print(f"Test R²    : {test_r2:.4f}")
print(f"Train RMSE : {train_rmse:.4f}")
print(f"Test RMSE  : {test_rmse:.4f}")
print(f"Train RPD  : {train_rpd:.4f}")
print(f"Test RPD   : {test_rpd:.4f}")

# -----------------------------
# Plots
# -----------------------------
fig, axes = plt.subplots(2, 2, figsize=(14,12))

# Training scatter
axes[0,0].scatter(y_train, y_pred_train, color='blue', alpha=0.6, label='Train Predicted')
z_train = np.polyfit(y_train, y_pred_train, 1)
p_train = np.poly1d(z_train)
axes[0,0].plot(y_train, p_train(y_train), "blue", linestyle='--', label="Train Best Fit")
axes[0,0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], "green", linestyle='-', label="Ideal Fit")
axes[0,0].set_title("Training Set: Predicted vs Actual")
axes[0,0].set_xlabel("Actual Values")
axes[0,0].set_ylabel("Predicted Values")
axes[0,0].legend()
axes[0,0].grid(True)

# Test scatter
axes[0,1].scatter(y_test, y_pred_test, color='red', alpha=0.6, label='Test Predicted')
z_test = np.polyfit(y_test, y_pred_test, 1)
p_test = np.poly1d(z_test)
axes[0,1].plot(y_test, p_test(y_test), "orange", linestyle='--', label="Test Best Fit")
axes[0,1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "green", linestyle='-', label="Ideal Fit")
axes[0,1].set_title("Test Set: Predicted vs Actual")
axes[0,1].set_xlabel("Actual Values")
axes[0,1].set_ylabel("Predicted Values")
axes[0,1].legend()
axes[0,1].grid(True)

# Training & Validation Loss
axes[1,0].plot(history.history['loss'], label='Training Loss', color='blue')
axes[1,0].plot(history.history['val_loss'], label='Validation Loss', color='orange')
axes[1,0].set_title("1D-CNN Training & Validation Loss")
axes[1,0].set_xlabel("Epochs")
axes[1,0].set_ylabel("MSE Loss")
axes[1,0].legend()
axes[1,0].grid(True)

# Confusion Matrix (Regression → Classification)
y_test_class = (y_test >= np.median(y_test)).astype(int)
y_pred_test_class = (y_pred_test >= np.median(y_test)).astype(int)
cm = confusion_matrix(y_test_class, y_pred_test_class)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Low Yield", "High Yield"])
disp.plot(ax=axes[1,1], cmap=plt.cm.Blues, values_format="d")
axes[1,1].set_title("Confusion Matrix")

plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'pandas'

In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

# GPU available কিনা দেখাবে
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# যদি GPU পাওয়া যায় তাহলে GPU নাম প্রিন্ট করবে
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU Detected:", gpus[0])
else:
    print("❌ No GPU detected. Check CUDA/cuDNN setup.")


TensorFlow version: 2.20.0
Num GPUs Available: 0
❌ No GPU detected. Check CUDA/cuDNN setup.
